In [1]:
# relative import ../build/fast_kinematics
import sys
sys.path.append('../build')
import numpy as np
import time
import fast_kinematics
import torch
import pytorch_kinematics as pk

In [2]:
N = 4096

In [7]:
model = fast_kinematics.FastKinematics("../kuka_iiwa.urdf",N,"lbr_iiwa_link_7")
qpos = np.random.rand(7*N)
qpos = qpos.astype(np.float32)

all_times = []
for _ in range(1000):
  start_time = time.time()
  poses = model.jacobian_mixed_frame(qpos)
  poses = poses.reshape(N,-1,6).transpose(0,2,1)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for jacobian_mixed_frame: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for jacobian_mixed_frame: 0.0003321645259857178 with std: 4.246280636118013e-05


In [4]:
chain = pk.build_serial_chain_from_urdf(open("../kuka_iiwa.urdf").read(), "lbr_iiwa_link_7")
d = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float32

chain = chain.to(dtype=dtype, device=d)
th = torch.rand(N, 7, dtype=dtype, device=d, requires_grad=True)
all_times = []
for _ in range(100):
  start_time = time.time()
  J = chain.jacobian(th)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for jacobian_mixed_frame: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for jacobian_mixed_frame: 0.019044153690338135 with std: 0.03550610747036887


In [10]:
th = torch.rand(N*7, dtype=torch.float32, device="cuda")
all_times = []
for _ in range(1000):
  start_time = time.time()
  model.forward_kinematics_pytorch(th)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for fk: {np.mean(all_times)} with std: {np.std(all_times)}")
# ---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
# Cell In [5], line 2
#       1 th = torch.rand(N*7, dtype=torch.float32, device="cuda")
# ----> 2 model.forward_kinematics_pytorch(th)

# TypeError: forward_kinematics_pytorch(): incompatible function arguments. The following argument types are supported:
#     1. (self: fast_kinematics.FastKinematics, t_angs: at::Tensor, block_size: int = 256) -> at::Tensor

# Invoked with: <fast_kinematics.FastKinematics object at 0x7f8d10dd8830>, tensor([0.3377, 0.2547, 0.7063,  ..., 0.2629, 0.3460, 0.0270], device='cuda:0')

Time taken for fk: 6.056308746337891e-06 with std: 6.163654710039213e-06


In [3]:
model = fast_kinematics.FastKinematics("../kuka_iiwa.urdf",N,"lbr_iiwa_link_7")
qpos = np.random.rand(7*N,1)
qpos = np.array(qpos, order='F', dtype=np.float32)

all_times = []
for _ in range(100):
  start_time = time.time()
  poses = model.do_nothing(qpos)
  # poses = poses.reshape(N,-1,6).transpose(0,2,1)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for jacobian_mixed_frame: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for jacobian_mixed_frame: 1.153707504272461e-05 with std: 1.2102644436211058e-05


In [6]:
qpos = qpos.reshape(N*7,1)
poses = model.forward_kinematics(qpos)
print(poses.shape)

(28672,)
